# Settings
"pip install Pyfhel" takes about 6mins on colab.

Python must have been compiled with C++17: g++>=6 | MSVC 2017+.

In [ ]:
!pip install Pyfhel

In [ ]:
import numpy as np
from Pyfhel import Pyfhel

In [ ]:
_r = lambda x: np.round(x, decimals=5)
float2ptxt = lambda x: HE.encodeFrac(np.array([x], dtype=np.float64))
float2ctxt = lambda x: HE.encryptFrac(np.array([x], dtype=np.float64))
ctxt2float = lambda x: HE.decryptFrac(x)[0]

# Setup (change coordinates here)
Server won't have access to plaintext client coordinate.

Client also won't have access to plaintext geofence coordinates.

In [ ]:
client_lat = 24.88
client_long = 181.7

In [ ]:
geofence_top = 24.89
geofence_down = 24.87
geofence_right = 181.9
geofence_left = 181.0

In [ ]:
client_lat *= 100
client_long *= 100

In [ ]:
geofence_top *= 100
geofence_down *= 100
geofence_right *= 100
geofence_left *= 100

# Client Side
Client specifies CKKS homomorphic encryption parameters. And encrypts its coordinate then send to server.

In [ ]:
# client side
HE = Pyfhel()
ckks_params = {
    'scheme': 'CKKS',
    'n': 2**14,
    'scale': 2**50,
    'qi_sizes': [60, 50, 50, 50, 60]
}
HE.contextGen(**ckks_params)

HE.keyGen()
HE.relinKeyGen()
# HE.rotateKeyGen()

In [ ]:
client_lat_ctxt = float2ctxt(client_lat)
client_long_ctxt = float2ctxt(client_long)

print(client_lat_ctxt, client_long_ctxt)

<Pyfhel Ciphertext at 0x7e65287262f0, scheme=ckks, size=2/2, scale_bits=50, mod_level=0> <Pyfhel Ciphertext at 0x7e6528725f80, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>


# Server Side

***RECEIVED HE ENCRYPTED COORDINATE FROM CLIENT***

Server calculates the intermediate result with encrypted client coordinate and geofence coordinates. Server then send obfuscated intermediate results with some additonal data(to check the integrity of the reply from client) to client.

In [ ]:
geofence_top_ptxt = float2ptxt(geofence_top)
geofence_down_ptxt = float2ptxt(geofence_down)
geofence_right_ptxt = float2ptxt(geofence_right)
geofence_left_ptxt = float2ptxt(geofence_left)

In [ ]:
a = -(client_lat_ctxt-geofence_top_ptxt)
b = client_lat_ctxt-geofence_down_ptxt
c = -(client_long_ctxt-geofence_right_ptxt)
d = client_long_ctxt-geofence_left_ptxt

In [ ]:
mul = lambda x, y: HE.align_mod_n_scale(~(x*y), a)[0]

In [ ]:
# compute the ciphertext for each combination
# doing this way is two minimize the mod_level
# combination x: binary string of x maps to the existence of a, b, c, d
# example: 11 -> 1011 -> a*c*d

one = float2ctxt(1)
ab = mul(a, b)
ac = mul(a, c)
ad = mul(a, d)
bc = mul(b, c)
bd = mul(b, d)
cd = mul(c, d)

all_combinations = [
    # 0000 0001 0010 0011 0100 0101 0110 0111
    one, d, c, cd, b, bd, bc, mul(bc, d),
    # 1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111
    a, ad, ac, mul(ac, d), ab, mul(ab, d), mul(ab, c), mul(ab, cd)
]
print(all_combinations)

[<Pyfhel Ciphertext at 0x7e652b0fb2e0, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>, <Pyfhel Ciphertext at 0x7e6528727e20, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>, <Pyfhel Ciphertext at 0x7e6528727ce0, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>, <Pyfhel Ciphertext at 0x7e65287cb470, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652b0fa980, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>, <Pyfhel Ciphertext at 0x7e6544869210, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e65287cb420, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e65287cb3d0, scheme=ckks, size=2/3, scale_bits=50, mod_level=2>, <Pyfhel Ciphertext at 0x7e652873d670, scheme=ckks, size=2/2, scale_bits=50, mod_level=0>, <Pyfhel Ciphertext at 0x7e65287cb380, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e65287269d0, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel C

In [ ]:
def append_message(message, message_combination, message_random_value, all_combinations, new_entry_combination):
  """
  function for adding an entry to the message that will be sent to client
  :param message: the message that will be sent to client, list of ciphertexts
  :param message_combination: list of integers, represent the combination of a, b, c, d of each message entry
  :param message_random_value: list of floats, represent the random value that is multiplied to each message entry
  :param all_combinations: list of ciphertexts, that map the combination index to ciphertext
  :param new_entry_combination: integer, represent the index of the combination for the new message entry
  :return: nothing, changes will be made to message, message_combination, message_random_value these three list
  """
  random_value = np.random.uniform(0.9,-0.9)
  # make random value in range [1, 0.1] or [-0.1, -1]
  # else the value may become too smal affecting the precision
  if 0.1>random_value>=0:
    random_value += 0.9
  elif 0>random_value>-0.1:
    random_value -= 0.9
  message_random_value.append(random_value)
  message_combination.append(new_entry_combination)
  random_ptxt = float2ptxt(random_value)
  message_ctxt = mul(all_combinations[new_entry_combination], random_ptxt)
  message.append(message_ctxt)
  return

In [ ]:
message = []
message_combination = []
message_random_value = []

# a
append_message(message, message_combination, message_random_value, all_combinations, 8)

# b
append_message(message, message_combination, message_random_value, all_combinations, 4)

# c
append_message(message, message_combination, message_random_value, all_combinations, 2)

# d
append_message(message, message_combination, message_random_value, all_combinations, 1)

for i in range(128):
  new_entry_combination = np.random.randint(16)
  append_message(message, message_combination, message_random_value, all_combinations, new_entry_combination)

print(message)
print(message_combination)
print(message_random_value)

[<Pyfhel Ciphertext at 0x7e65287cbe20, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652b0fbab0, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e65287cbe70, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652873d800, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652871f740, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e654486a840, scheme=ckks, size=2/3, scale_bits=50, mod_level=2>, <Pyfhel Ciphertext at 0x7e652be29b20, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652873c6d0, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e652873ccc0, scheme=ckks, size=2/4, scale_bits=50, mod_level=3>, <Pyfhel Ciphertext at 0x7e652873de90, scheme=ckks, size=2/2, scale_bits=50, mod_level=1>, <Pyfhel Ciphertext at 0x7e65287246d0, scheme=ckks, size=2/4, scale_bits=50, mod_level=3>, <Pyfhel C

# Client Side

***RECEIVED OBFUSCATED ENCRYPTED INTERMEDIATE RESULT WITH SOME ADDTIONAL DATA FROM SERVER***

Client decrypt all of them with secret key and send the signed(sign function) version to server.

Idealy, client will only be able to fake the data if client guessed the combinations and random value of all the addtional data correctly.

In [ ]:
decrypted_message = []
signed_decrypted_message = []
for cyphertext in message:
  decrypted_float = ctxt2float(cyphertext)
  decrypted_message.append(decrypted_float)
  signed_decrypted_message.append(1 if decrypted_float>0 else -1)

print(decrypted_message)
print(signed_decrypted_message)
# signed_decrypted_message[0] = 1

[-0.872198333796081, 0.4591387939512981, 14.2925891679828, -53.45007291647742, -8.361247900515993, -1381.8762338246913, -0.570284943008293, -4.630309820840839, -44.53597430737135, 7.5766963855284395, -571.6874963448322, -0.33069626200078484, 0.9681753698568781, 39.98186412071772, 51.60404680620102, 0.19728921107584443, -15.051112490102167, 6.787807710473199, 0.8987352364161556, -0.7798378127828842, -16.341512582448527, -58.510957341182255, -0.7109732029293873, -9.436125918313842, -18.100794822355105, -28.251682155593414, -511.2767162540637, 0.21023743461115796, 18.002503801649755, 41.409127083995884, -0.2998292957775682, -0.1026556280832685, 12.68691502581225, 0.9345933577267389, 16.866491168272006, 18.00300108808639, -65.85414841054731, 9.530010351794672, 0.5568642309781339, -16.278463167362087, 11.59310571216402, -11.660395901023389, 19.011121575281773, -18.582327106189947, 0.7151588141698548, -16.39178674717831, -1018.537728307921, -0.4415238589570059, 0.7122932780086498, -25.399932

# Server Side (Result)
***RECEIVED SIGNED(SIGN FUNCTION) PLAINTEXT INTERMEDIATE RESULT***

Server first check the integrity of the data from client with "the additional data" (make sure client didn't fake it).

If the data is correct, server then can check whether client is inside the geofence or not.

In [ ]:
for i in range(len(signed_decrypted_message)):
  assert(signed_decrypted_message[i]==1 or signed_decrypted_message[i]==-1)

decrypted_abcd = []
for i in range(4):
  if message_random_value[i]>0:
    decrypted_abcd.append(signed_decrypted_message[i])
  else:
    decrypted_abcd.append(-signed_decrypted_message[i])

print(decrypted_abcd)

decrypted_combinations = []
for i in range(16):
  current_decrypted_combination = 1
  for j in range(3, -1, -1):
    if (i>>j)&1:
      current_decrypted_combination *= decrypted_abcd[3-j]
  decrypted_combinations.append(current_decrypted_combination)
print(decrypted_combinations)

for i in range(4, len(signed_decrypted_message)):
  # print(decrypted_combinations[message_combination[i]] * message_random_value[i] * signed_decrypted_message[i])
  # print(message_random_value[i])
  # print(signed_decrypted_message[i])
  # print(decrypted_message[i])
  # print(message_combination[i])

  assert decrypted_combinations[message_combination[i]] * message_random_value[i] * signed_decrypted_message[i] > 0, "client modified the data or location too near the edge"
print("We can assume the client didn't fake the data.")

if decrypted_abcd[0]==decrypted_abcd[1]==-1 or decrypted_abcd[2]==decrypted_abcd[3]==-1:
  print("Not possible in real world, something went wrong")
elif decrypted_abcd[0]==decrypted_abcd[1]==decrypted_abcd[2]==decrypted_abcd[1]==1:
  print("Client in geofence, Location verified. Service granted.")
else:
  print("Client not in geofence. Location not verified. Service denied.")


[1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
We can assume the client didn't fake the data.
Client in geofence, Location verified. Service granted.
